In [4]:
import requests
from json import loads
import os
import pandas as pd
import numpy as np
import collections

In [22]:
# Register for a free API key https://etherscan.io/myapikey
API_KEY = "REPLACEME"
SRC = "0x5149Aa7Ef0d343e785663a87cC16b7e38F7029b2".lower()
DST = "0x2Fd3F2701ad9654c1Dd15EE16C5dB29eBBc80Ddf".lower()

# time of first txn out of SRC
START_TIME = 1590927652 - 300
END_TIME = 1590933273

# transaction values are stored as ints (original value * 1e18)
MULTIPLIER = 1e18

In [2]:
def get_txn(addr, ignore_cache=False):
    cache_file = f'cache/{addr}'
    
    if not ignore_cache and os.path.isfile(cache_file):
        with open(cache_file, "r") as f:
            return loads(f.read())["result"]
    
    r = requests.get(f'https://api-ropsten.etherscan.io/api?module=account&action=txlist&address={addr}&tag=latest&apikey={API_KEY}',
                    headers={
                        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36"
                    })
    data = loads(r.text)
    if data["message"] != "OK":
        print(data)
        raise Exception("Bad response")
    
    with open(cache_file, "w+") as f:
        f.write(r.text)
    
    return data["result"]
    

Extract all transactions made by the source and destination addresses

In [4]:
src_txn = pd.DataFrame(get_txn(SRC))
dst_txn = pd.DataFrame(get_txn(DST))

src_txn["timeStamp"] = src_txn["timeStamp"].astype("int64")
dst_txn["timeStamp"] = dst_txn["timeStamp"].astype("int64")

Extract all addresses that the src address feeds into, and all addresses that feed into the dst address. These addresses are treated as "known tumbler" addresses

In [5]:
tumbler_in_addr = src_txn[(src_txn["from"] == SRC) & (src_txn["timeStamp"] > START_TIME) & (src_txn["timeStamp"] < END_TIME)]["to"]
tumbler_out_addr = dst_txn[(dst_txn["to"] == DST) & (dst_txn["timeStamp"] > START_TIME) & (dst_txn["timeStamp"] < END_TIME)]["from"]

In [6]:
tumbler_addr = list(set(tumbler_in_addr.tolist() + tumbler_out_addr.tolist()))

txn_acc = []
for addr in tumbler_addr:
    txn_acc += get_txn(addr)

txn = pd.DataFrame(txn_acc)

In [7]:
txn

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations
0,7980980,1590590977,0xb2c0d083a0943290906b1476516c5b657845723a35ff...,33110063,0x460adfcb5fe3978be8e1958fa088c343b81486efda6d...,60,0x81b7e08f65bdf5648606c89998a9cc8164397647,0xd8340e8907f2f10fad701cf632adb2504d3a8f51,1000000000000000000,21000,16,0,1,0x,,7854918,21000,55641
1,7980980,1590590977,0x0c98280db4862a95f82e243871af35332cb41505f59a...,33110064,0x460adfcb5fe3978be8e1958fa088c343b81486efda6d...,63,0x81b7e08f65bdf5648606c89998a9cc8164397647,0xd8340e8907f2f10fad701cf632adb2504d3a8f51,1000000000000000000,21000,16,0,1,0x,,7917918,21000,55641
2,7980981,1590591022,0x1f955329e2e222ac061ca6b3866882773c83fa8f4af2...,33110065,0xba5e88bbd7e2744bdc4a5f54466e1510f497c636b2cc...,30,0x81b7e08f65bdf5648606c89998a9cc8164397647,0xd8340e8907f2f10fad701cf632adb2504d3a8f51,1000000000000000000,21000,16,0,1,0x,,7886819,21000,55640
3,7980981,1590591022,0x54a7d9d89ecd2770b9b8f17c6cf2158ec9177502c4af...,33110067,0xba5e88bbd7e2744bdc4a5f54466e1510f497c636b2cc...,34,0x81b7e08f65bdf5648606c89998a9cc8164397647,0xd8340e8907f2f10fad701cf632adb2504d3a8f51,1000000000000000000,21000,16,0,1,0x,,7977815,21000,55640
4,7980982,1590591045,0x395c3da9caf8e4894627514a6ec21f2037855a0ecceb...,33110071,0x8b1ff54cae23744629f7dd00c2c69832e07ba35481dd...,30,0x81b7e08f65bdf5648606c89998a9cc8164397647,0xd8340e8907f2f10fad701cf632adb2504d3a8f51,1000000000000000000,21000,16,0,1,0x,,7640289,21000,55639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65123,8029130,1591270883,0x755b9cb1c43bcf8a539b6ff6b0f08a105ce313e52926...,106,0x69ab73bf299ab69170c00c4e9eb227793fb5716c096e...,21,0x98c6fe0e5407643d8623be4e7db6c4799db58865,0xed281881446e9140119dc617ed7e8e5e5a7ee182,2700000000000000000,100000,1,0,1,0x,,4748450,21000,7488
65124,8029132,1591270906,0x30190dd679329d6be359148db46748b6155f80d097d4...,100,0xb66bde82c61cc4c11622dbd60ae837128d894743a680...,51,0xed281881446e9140119dc617ed7e8e5e5a7ee182,0x61e4caea99dbd3d071ef38d5592dbecc3a284a37,1900000000000000000,100000,1,0,1,0x,,6113825,21000,7486
65125,8029132,1591270906,0xf32d97d34fb0d01cc9d26c9dfe331315e37e012cc303...,88,0xb66bde82c61cc4c11622dbd60ae837128d894743a680...,72,0xe512b7d0b293d1eddd8c03d898fcedc7d5c639f5,0xed281881446e9140119dc617ed7e8e5e5a7ee182,8300000000000000000,100000,1,0,1,0x,,6554825,21000,7486
65126,8029142,1591271024,0xfdd4a7b157bd4502b7b0dbb224f534ad6fa3268821af...,84,0xe2bf5fcbcb804034394cb73c1e26c1f52c89c747cf60...,96,0x0150530ae45f8529d9493fb1ef4b8dc79482aac0,0xed281881446e9140119dc617ed7e8e5e5a7ee182,3100000000000000000,100000,1,0,1,0x,,6529401,21000,7476


Extract all transactions that target tumbler addresses and are NOT from a tumbler address

In [8]:
tumbler_rx_txn = txn[txn["to"].isin(tumbler_addr) & (~txn["from"].isin(tumbler_addr))]

Compute how much each individual address has sent into the tumbler network

In [9]:
d = collections.defaultdict(lambda: 0)

for i, t in tumbler_rx_txn.iterrows():
    d[t["from"]] += int(t["value"])

In [10]:
inps = list(dict(d).items())
inps.sort(key=lambda x: x[1])

In [11]:
inps[-20:]

[('0xa8e0fd25440cf751f9b06b1df131c01fcebb716b', 161100000000000000000),
 ('0x6469e8c31bbfdd74d79760778b801fb3d81aed55', 161100000000000000000),
 ('0xd176fe27713aa3c269bae5635c39ad3092013b39', 162500000000000000000),
 ('0x72bff7e51a5b0effb6cbd92c034898b4363cb9c5', 162900000000000000000),
 ('0xfee255ec39c110bff6b7dbac97daf5977196369e', 163000000000000000000),
 ('0x5700df105fde72837c85341a85a28ac8b52ef175', 165200000000000000000),
 ('0xe62a99f65e602719db9ef029a64f00b5235651d9', 166900000000000000000),
 ('0x589f0bc0dc3639c69785d9fdf08b5b7395284370', 167400000000000000000),
 ('0xd2e20f324a612a0a7d1f14986fbd9d835bdd1331', 167900000000000000000),
 ('0x0b512200b606d69978248bd2f814620a883699f6', 171300000000000000000),
 ('0xe480a848af5c9dcd755dee9f0336355ed2f2016b', 172100000000000000000),
 ('0x8f0988a77961fa5aebc150ed647f1cf1ebc1b0ad', 174400000000000000000),
 ('0x1bcce97cf90fa566631dafbdd92cc0df91cd7a86', 178800000000000000000),
 ('0xf589f7405f0be5102e073ed92fab170ac8feaac9', 1814000000000000

0x4c5e179bbc6d393affb72018f9bba4b3cee6de65 is the address that has sent the next largest amount of eth into the tumbler, works when submitted as the flag

In [10]:
wallet_txn = pd.DataFrame(get_txn("0x4c5e179bbc6d393affb72018f9bba4b3cee6de65"))
wallet_txn["timeStamp"] = wallet_txn["timeStamp"].astype("int64")
wallet_txn

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,gasPrice,isError,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations
0,8002003,1590921689,0x89a91bff7355e355bd9be9cd254b5195f5a7f6c2de75...,0,0x5bc94bdffc5491a58175a5077a92b0b85ebc9911abe5...,3,0x5dbc9da0317c550460d501742294ac9d07d7054a,0x4c5e179bbc6d393affb72018f9bba4b3cee6de65,4999900000000000000,100000,1000000000,0,1,0x,,195226,21000,44665
1,8002004,1590921692,0xc9e80a9be0e5e4e279ad3af72eec2704cc18bf112cfc...,0,0x89a8c29e7d1a0f9a85e60c0b98f875313ad0466361fc...,2,0xa8325fb2c1c15da2396ad77580c136d13578dd30,0x4c5e179bbc6d393affb72018f9bba4b3cee6de65,4999900000000000000,100000,1000000000,0,1,0x,,69006,21000,44664
2,8002004,1590921692,0x17cf97273309f45d04e227ca7d8ad2fe62186c7f3149...,0,0x89a8c29e7d1a0f9a85e60c0b98f875313ad0466361fc...,3,0x8c9e8951f64900585572e8eb8c4dc8c0bb38dce5,0x4c5e179bbc6d393affb72018f9bba4b3cee6de65,4999900000000000000,100000,1000000000,0,1,0x,,90006,21000,44664
3,8002004,1590921692,0x38e2c671d63932594adf437bb1e09ce9b7371259123a...,0,0x89a8c29e7d1a0f9a85e60c0b98f875313ad0466361fc...,4,0xd3c7f4cf33ed62e5d52eb7e6e9d9161ae1a164a2,0x4c5e179bbc6d393affb72018f9bba4b3cee6de65,4999900000000000000,100000,1000000000,0,1,0x,,111006,21000,44664
4,8002005,1590921695,0x1a1451ac85dbd990f98bf7ca2c7939107285240be2fc...,0,0x07649f39eba3d3749f5d47e6f8b191d5ad775020e6fb...,0,0x2458b57bfc409e3660dedc7d0ee954289572b1d2,0x4c5e179bbc6d393affb72018f9bba4b3cee6de65,4999900000000000000,100000,1000000000,0,1,0x,,21000,21000,44663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,8029108,1591270656,0xc7d469535b76ff7b400f75dcefc01970da8f39ae3895...,229,0x8776d6302a9864b67a31daba3b69a3b11f271a8fa83d...,69,0x4c5e179bbc6d393affb72018f9bba4b3cee6de65,0xe1399b5256fafc2b48c2d0e6d7f39007bc9be8a4,5400000000000000000,100000,1,0,1,0x,,5925987,21000,17560
365,8029118,1591270763,0x9618a0fa91a9ba5cf527b329e8020695db9a751b4a66...,92,0x322a618836aa463da1fb2f61aadbc196f59db994827a...,122,0x2b3bf8b02dd2476eefa18aa63da65bd8c03a0147,0x4c5e179bbc6d393affb72018f9bba4b3cee6de65,5400000000000000000,100000,1,0,1,0x,,7143251,21000,17550
366,8029142,1591271024,0x45a82330a7772997ffe2f459a9d70f1a514abe5d54fb...,99,0xe2bf5fcbcb804034394cb73c1e26c1f52c89c747cf60...,159,0x97fc2f5d7b4e05bc544f5f4053fd2e72af6c3c5e,0x4c5e179bbc6d393affb72018f9bba4b3cee6de65,900000000000000000,100000,1,0,1,0x,,7879019,21000,17526
367,8029145,1591271120,0x1c2364b5aa9e9bb9a569572e1a677241382232f3bb9c...,230,0x034c7b031bbb5a8d372a88a297f669804be1bcdcfbf2...,124,0x4c5e179bbc6d393affb72018f9bba4b3cee6de65,0x2a621252a0ee1c8655b09eea7905d87be28fe9e3,6800000000000000000,100000,1,0,1,0x,,7318324,21000,17523


In [23]:
out_vals = wallet_txn[(wallet_txn["from"] == "0x4c5e179bbc6d393affb72018f9bba4b3cee6de65") & (wallet_txn["timeStamp"] > START_TIME) & (wallet_txn["timeStamp"] < END_TIME)]["value"]
total_out = sum(map(int, out_vals))
total_out / MULTIPLIER

461.9032581447249

We know that the attacker:
* Put 435 eth from Wallet A into the tumbler network
* Took out 881 eth from Wallet C from the tumbler network

The difference, 881 - 435 = 446 matches the amount put in by `0x4c5e179bbc6d393affb72018f9bba4b3cee6de65` (assuming the difference is from fees)